In [1]:
import os
import numpy as np
from PIL import Image

# Rutas de las carpetas y ratios de mezcla
spectrogram_folders = {
    ('./OfficialSpectrogramsMelAUG/train/crackle', './OfficialSpectrogramsMelAUG/train/normal', './OfficialSpectrogramsMelAUG/train/crackle'): 0.5,
    ('./OfficialSpectrogramsMelAUG/train/wheeze', './OfficialSpectrogramsMelAUG/train/normal', './OfficialSpectrogramsMelAUG/train/wheeze'): 1,
    ('./OfficialSpectrogramsMelAUG/train/crackle', './OfficialSpectrogramsMelAUG/train/wheeze', './OfficialSpectrogramsMelAUG/train/crackle_wheeze'): 1
}
destination_folder = './OfficialSpectrogramsMelAUG/train'

# Función para mezclar espectrogramas y guardar
def mixup_and_save(folder1, folder2, output_folder, ratio):
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Obtener lista de espectrogramas originales (excluir los aumentados y los ya mezclados)
    spectrograms1 = [f for f in os.listdir(folder1) if not (f.startswith(('pitch_shift', 'random_shift', 'volume_adjust', 'time_stretch', 'mixup')))]
    spectrograms2 = [f for f in os.listdir(folder2) if not (f.startswith(('pitch_shift', 'random_shift', 'volume_adjust', 'time_stretch', 'mixup')))]

    # Seleccionar aleatoriamente espectrogramas para mezclar según el ratio
    num_samples = int(len(spectrograms1) * ratio)
    selected_spectrograms1 = np.random.choice(spectrograms1, num_samples, replace=False)

    # Realizar mixup y guardar
    for spectrogram1 in selected_spectrograms1:
        # Seleccionar aleatoriamente un espectrograma de folder2 para mezclar
        spectrogram2 = np.random.choice(spectrograms2)
        
        # Cargar espectrogramas
        spec1 = Image.open(os.path.join(folder1, spectrogram1))
        spec2 = Image.open(os.path.join(folder2, spectrogram2))

        # Realizar mixup
        alpha = np.random.beta(0.4, 0.4)
        mixed_spec = Image.blend(spec1, spec2, alpha)

        # Guardar espectrograma mezclado
        mixed_spec_filename = f"mixup_{spectrogram1[:-4]}_{spectrogram2[:-4]}.png"
        mixed_spec_path = os.path.join(output_folder, mixed_spec_filename)
        mixed_spec.save(mixed_spec_path)

# Realizar mixup para cada configuración
for folders, ratio in spectrogram_folders.items():
    folder1, folder2, output_folder = folders
    mixup_and_save(folder1, folder2, output_folder, ratio)